In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

import warnings
warnings.filterwarnings('ignore')

tf.__version__

In [ ]:
df = pd.read_csv('./../data/imdbtop1000/imdb_data.csv', sep='\t')
df = df.rename(columns={'User Votes': 'Votes',
                        'Imdb Rating': 'Rating',
                       'Gross(in Million Dollars)': 'Earnings',
                       'Runtime(Minutes)' : 'Runtime'})
df.head()

In [ ]:
dataframe = df[['Votes', 'Earnings', 'Rating']]
#It is very important to normalise the input features in a proper range
#It helps in avoiding very large calculations
dataframe['Votes'] = dataframe['Votes'] / 1000000
dataframe['Earnings'] = dataframe['Earnings'] / 100
dataframe.describe()

In [ ]:
#There are 73 (1000 - 927) rows with NaN/nan values
#Drop those rows
dataframe.dropna(inplace=True)

In [ ]:
#Empty dataframe confirms abscence of rows with nan/NaN
dataframe[dataframe.Earnings.isnull()]

In [ ]:
from tensorflow.keras import layers, Sequential, optimizers

In [ ]:
#Building Feature Column
feature_columns = []

#Creating a numerical feature columns to represent Rating and Earning features
votes = tf.feature_column.numeric_column("Votes")
feature_columns.append(votes)

earnings = tf.feature_column.numeric_column("Earnings")
feature_columns.append(earnings)

#Converting the list of features into a layer that will be fed to the network
feature_layer = layers.DenseFeatures(feature_columns)

In [ ]:
def build_model(lr, feature_layer):
    #initialise model :: Sequential Model
    model = Sequential()
    
    #adding feature layer
    model.add(feature_layer)
    
    #Add layers to the model
    model.add(layers.Dense(units=1, input_shape=(2,)))
    
    #Compile model
    #Configure training to minimize the model's mean squared error.
    model.compile(optimizer=optimizers.SGD(lr=lr),
                    loss="mean_squared_error")
    
    return model

In [ ]:
def train(model, dataset, label_name, epochs, batch_size):
    #Here, we're passing
    # every column in the dataset. Note that the feature_layer will filter
    # away most of those columns, leaving only the desired columns and their
    # representations as features.
    features = {name:np.array(value) for name, value in dataset.items()}
    label = np.array(features.pop(label_name)) 
    
    
    #Feeding the model training data
    history = model.fit(features,
                        label,
                        batch_size=batch_size,
                        epochs=epochs)
    
    return model, history

In [ ]:
learning_rate = 0.1
epochs = 50
batch_size = len(df)

label = "Rating"

model = build_model(learning_rate, feature_layer)

trained_model, hist = train(model, dataset=dataframe, label_name=label, epochs=epochs, batch_size=batch_size)


In [ ]:
trained_model.get_weights()

In [ ]:
LOSS = pd.DataFrame(hist.history)['loss']

plt.figure(figsize=(8,6))
plt.plot(LOSS, label='LOSS')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()